# モチーフは機能を示唆する

## 問題文

あるタンパク質モチーフは、その多様な形を考慮するために次のような簡略表記で表されます：

- `[XY]` は「X または Y のいずれか」
- `{X}` は「X 以外の任意のアミノ酸」

例えば、N-グリコシル化モチーフは `N{P}[ST]{P}` と表されます。  
これは、「N の後に P 以外の任意のアミノ酸、次に S または T、その後に P 以外の任意のアミノ酸」という4文字の連続配列を意味します。

## データ取得方法

UniProt データベースでは、タンパク質のアクセス ID（`uniprot_id`）を使ってその完全な情報を見ることができます：

- タンパク質の説明ページ:  
  `http://www.uniprot.org/uniprot/uniprot_id`

- FASTA 形式の配列データ取得:  
  `http://www.uniprot.org/uniprot/uniprot_id.fasta`

例：`B5ZC00` のデータは以下で閲覧できます：

- 説明ページ: http://www.uniprot.org/uniprot/B5ZC00  
- FASTA: http://www.uniprot.org/uniprot/B5ZC00.fasta

## 入力

最大15個の UniProt タンパク質アクセス ID が与えられます。

例：
```
A2Z669
B5ZC00
P07204_TRBM_HUMAN
P20840_SAG1_YEAST
```

## 出力

各タンパク質について、N-グリコシル化モチーフ `N{P}[ST]{P}` を含む場合は以下を出力します：

- 該当するアクセス ID
- そのモチーフが出現するすべての位置（1始まり、空白区切り）

## 出力例

```
B5ZC00
85 118 142 306 395
P07204_TRBM_HUMAN
47 115 116 382 409
P20840_SAG1_YEAST
79 109 135 248 306 348 364 402 485 501 614
```

## 備考

- FASTA ファイルからは、ヘッダー（`>`で始まる行）を除き、配列本体のみを使用してください。
- 正規表現によるパターンマッチングが必要です。
- 出力する位置は 1-indexed（最初の文字を 1 として数える）である点に注意してください。


In [1]:
import requests
def request_fasta(ids:list[str]):
    fastas = []
    for id in ids:
        url = f"https://www.uniprot.org/uniprot/{id}.fasta"
        response = requests.get(url)
        if response.status_code == 200:
            fasta = response.text
            fastas.append(fasta)
        else:
            print(f"Error in {id}: {response.status_code}")
    return fastas

# HTTP ステータスコード まとめ from GPT

[簡単なまとめサイト](https://white-link.com/sem-plus/http-statuscode/)

## ✅ 成功（2xx）

| ステータスコード | 意味                      | 説明                                       |
|------------------|---------------------------|--------------------------------------------|
| 200 OK           | 成功                      | リクエストが正常に処理された               |
| 201 Created      | 作成成功                  | 新しいリソースが作成された（POSTなど）     |
| 204 No Content   | 成功（内容なし）          | 処理成功したが返す内容がない               |

---

## 🔁 リダイレクト（3xx）

| ステータスコード | 意味                      | 説明                                       |
|------------------|---------------------------|--------------------------------------------|
| 301 Moved Permanently | 恒久的に移動         | リソースが別のURLに移動した（恒久）        |
| 302 Found        | 一時的な移動              | 一時的に別のURLにリダイレクトされる        |
| 304 Not Modified | 変更なし                  | キャッシュされたデータで問題なし           |

---

## ⚠️ クライアントエラー（4xx）

| ステータスコード | 意味                      | 説明                                       |
|------------------|---------------------------|--------------------------------------------|
| 400 Bad Request  | 不正なリクエスト          | リクエスト形式やパラメータに誤りがある     |
| 401 Unauthorized | 未認証                    | 認証が必要、または認証情報が無効           |
| 403 Forbidden    | 禁止                      | アクセス権がない                           |
| 404 Not Found    | 見つからない              | リソースが存在しない                       |
| 429 Too Many Requests | リクエスト過多       | 一定時間内にリクエストが多すぎる           |

---

## ❌ サーバーエラー（5xx）

| ステータスコード | 意味                      | 説明                                       |
|------------------|---------------------------|--------------------------------------------|
| 500 Internal Server Error | サーバー内部エラー | サーバー側で予期しないエラーが発生         |
| 502 Bad Gateway  | 不正なゲートウェイ         | 上流サーバーからの無効な応答               |
| 503 Service Unavailable | 利用不可           | サーバーが一時的に処理不能（メンテ中など）|



In [2]:
def read_fasta(fastas,keys):
    sequences = {}
    header = None
    seq = []
    for fasta, key in zip(fastas, keys):
        fasta = fasta.splitlines()
        for line in fasta:
            if line.startswith('>'):
                if header is not None:
                    sequences[header] = ''.join(seq)
                header = line  # Remove '>'
                seq = []
            else:
                seq.append(line)
        if header is not None:
            sequences[key] = ''.join(seq)  # Add the last sequence
    return sequences

In [3]:
import re
def search_motif(sequences, motif):
    pos = re.finditer(motif, sequences)
    results = []
    for match in pos:
        start = match.start() + 1
        results.append(start)        
    if results:
        return '\t'.join(map(str, results))
    return None

In [4]:
def main(ids:list[str], motif:str):
    fastas = request_fasta(ids)
    sequences = read_fasta(fastas, ids)
    for key, seq in sequences.items():
        positions = search_motif(seq, motif)
        if positions:
            print(key)
            print(positions)

In [5]:
motif = "N[^P][ST][^P]"
ids = ["A2Z669", "B5ZC00", "P07204_TRBM_HUMAN", "P20840_SAG1_YEAST"]
main(ids, motif)

Error in P07204_TRBM_HUMAN: 400
Error in P20840_SAG1_YEAST: 400
B5ZC00
85	118	142	306	395
